<a href="https://colab.research.google.com/github/Arumat546/nico_cls/blob/main/%E3%83%86%E3%82%AD%E3%82%B9%E3%83%88%E5%89%8D%E5%87%A6%E7%90%86.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%cd /content/drive/My Drive/Google Colab/niconico_dataset_add_smid

/content/drive/My Drive/Google Colab/niconico_dataset_add_smid


In [3]:
# 乱数シードの固定
import os
import random
import numpy as np
import torch

SEED_VALUE = 10  # これはなんでも良い
os.environ['PYTHONHASHSEED'] = str(SEED_VALUE)
random.seed(SEED_VALUE)
np.random.seed(SEED_VALUE)
torch.manual_seed(SEED_VALUE)  # PyTorchを使う場合

In [ ]:
#tsv出力
import pandas as pd
import sqlite3
import csv

i=0
df = pd.DataFrame(index=[], columns=['content','category'])

con = sqlite3.connect("2525.db")
c = con.cursor()
c.execute("select content,category from comment5 inner join video on comment5.video_id=video.video_id where length(content)>=8 and category='動物'")
for row in c: # rowはtuple
    #print(row[0], row[1])
    df.loc[i]=[row[0], row[1]]
    i+=1

df.head()
#データをシャッフル
df = df.sample(frac=1, random_state=32).reset_index(drop=True)

#データ前処理
df['content'] = df['content'].map(clean_text)
df['content'] = df['content'].map(normalize)
#df['content'] = df['content'].map(labeling)

#6文字以下の文章の除去
df['content'] = df['content'].astype('str')
mask = (df['content'].str.len() >= 7)
df = df.loc[mask]
#print(df)


#取り出すデータ数指定
df[:200].to_csv('/content/drive/My Drive/Google Colab/niconico_dataset_add_smid/DATASET/ppp.tsv', sep='\t', index=False, header=None)
#print(df[:len].shape)
#df.to_csv('/content/entame.tsv', sep='\t', index=False)
con.commit()
con.close()

In [ ]:
#前処理実行
import pandas as pd
import csv

df = pd.read_table("seizi.tsv", encoding='utf-8')
#print(df)
#print(df['content'][8])

#前処理
df['content'] = df['content'].map(clean_text)
df['content'] = df['content'].map(normalize)
df['category'] = df['category'].map(labeling)

#4文字以下の文章の除去
df['content'] = df['content'].astype('str')
mask = (df['content'].str.len() > 4)
df = df.loc[mask]
print(df)


df.to_csv("seizi8000.tsv", sep='\t', index=None, encoding='utf-8')
#print(df)

In [3]:
#ラベルの数値化
def labeling(text):
    text = re.sub(r'  ゲーム', ' 2', text)
    #text = re.sub(r' ', '  1', text)
    #text = re.sub(r' ', '  2', text)
    return text

## **記号除去**

In [4]:
#記号除去
!pip install neologdn
!pip install emoji
import string
import re
import neologdn
import emoji
def clean_text(text):
  text = neologdn.normalize(text)#全角・半角の統一  
  text = re.sub(r'https?://[\w/:%#\$&\?\(\)~\.=\+\-]+', '',text)  # URLの除去 
  text = re.sub(r'[!-/:-@[-`{-~]', r' ', text)# 半角記号の置換
  text = re.sub(u'[■-♯]', ' ', text)#全角記号の置換
  text = re.sub(r'[ヽ「」ω仝و∵ξν〃━➑✪█◢†彡ヾ∀∇≦°д゛※‥…→←↑・↓]', '',text)    #  特殊文字の除去
  text = re.sub(r'、', ' ',text)#、。の空白化
  text = re.sub(r'。', ' ',text)
  text = re.sub(r'　', ' ',text)  # 全角空白の半角化
  text = ''.join(['' if c in emoji.UNICODE_EMOJI else c for c in text])#絵文字除去
  
  return text

"""
def clean_text(text):
    #replaced_text = '\n'.join(s.strip() for s in text.splitlines()[2:] if s != '')  # skip header by [2:]
    replaced_text = text.lower()
    replaced_text = re.sub(r'[【】]', '', text)       # 【】の除去
    replaced_text = re.sub(r'[（）()]', '', replaced_text)     # （）の除去
    replaced_text = re.sub(r'[［］\[\]]', '', replaced_text)   # ［］の除去
    replaced_text = re.sub(r'[「」｢｣『』]', '', replaced_text)     # 「」の除去
    replaced_text = re.sub(r'[{}｛｝]', '', replaced_text)    #  {}の除去
    replaced_text = re.sub(r'[；;＾^￥：:、。・￥!！"#$%&=~”＃＄％＆’|*.※‥…→←↑↓]', '', replaced_text)    #  その他記号の除去   
    replaced_text = re.sub(r'https?:\/\/.*?[\r\n ]', '', replaced_text)  # URLの除去   
    replaced_text = re.sub(r'[ヽω仝و∵ξν〃━➑✪█◢†彡ヾ∀∇≦°д゛]', '', replaced_text)    #  特殊文字の除去
    replaced_text = re.sub(r'　', '', replaced_text)  # 全角空白の除去

    return replaced_text    
"""

     |████████████████████████████████| 61kB 3.1MB/s 
  Created wheel for neologdn: filename=neologdn-0.4-cp36-cp36m-linux_x86_64.whl size=186915 sha256=42116a5ec595d34c70c2e8c5ddda8f1e63e59defe32f00d6431f29d6d917bd6e
  Stored in directory: /root/.cache/pip/wheels/b3/6f/d4/c132b4e7aef22019e307e7673d97010644c9c15f28c0d0b018
Successfully built neologdn
     |████████████████████████████████| 51kB 2.8MB/s 
  Created wheel for emoji: filename=emoji-0.6.0-cp36-none-any.whl size=49717 sha256=7b70b4014e0e1167fe39f085999c3b9be0b39282aeb03449c4aa0bfb61e446dc
  Stored in directory: /root/.cache/pip/wheels/46/2c/8b/9dcf5216ca68e14e0320e283692dce8ae321cdc01e73e17796
Successfully built emoji


'\ndef clean_text(text):\n    #replaced_text = \'\n\'.join(s.strip() for s in text.splitlines()[2:] if s != \'\')  # skip header by [2:]\n    replaced_text = text.lower()\n    replaced_text = re.sub(r\'[【】]\', \'\', text)       # 【】の除去\n    replaced_text = re.sub(r\'[（）()]\', \'\', replaced_text)     # （）の除去\n    replaced_text = re.sub(r\'[［］\\[\\]]\', \'\', replaced_text)   # ［］の除去\n    replaced_text = re.sub(r\'[「」｢｣『』]\', \'\', replaced_text)     # 「」の除去\n    replaced_text = re.sub(r\'[{}｛｝]\', \'\', replaced_text)    #  {}の除去\n    replaced_text = re.sub(r\'[；;＾^￥：:、。・￥!！"#$%&=~”＃＄％＆’|*.※‥…→←↑↓]\', \'\', replaced_text)    #  その他記号の除去   \n    replaced_text = re.sub(r\'https?:\\/\\/.*?[\r\n ]\', \'\', replaced_text)  # URLの除去   \n    replaced_text = re.sub(r\'[ヽω仝و∵ξν〃━➑✪█◢†彡ヾ∀∇≦°д゛]\', \'\', replaced_text)    #  特殊文字の除去\n    replaced_text = re.sub(r\'\u3000\', \'\', replaced_text)  # 全角空白の除去\n\n    return replaced_text    \n'

In [ ]:
text="d　a n,c、e。D.A!N！C?E？"
print(clean_text(text))

d a n c e D A N C E 


## **基本前処理**

In [5]:
# -*- coding: utf-8 -*-
import re
import unicodedata
import nltk
from nltk.corpus import wordnet

#まとめて処理
def normalize(text):
    #normalized_text = normalize_unicode(text)
    normalized_text = normalize_number(text)
    normalized_text = lower_text(normalized_text)
    normalized_text = reduce_seq(normalized_text)
    normalized_text = reduce_w(normalized_text)
    return normalized_text


#大文字→小文字
def lower_text(text):
    return text.lower()

#半角全角統一
#def normalize_unicode(text, form='NFKC'):
 #   normalized_text = unicodedata.normalize(form, text)
  #  return normalized_text

#数字列の置換
def normalize_number(text):
    """
    pattern = r'\d+'
    replacer = re.compile(pattern)
    result = replacer.sub('0', text)
    """
    # 連続した数字を0で置換
    replaced_text = re.sub(r'\d+', '0', text)
    return replaced_text

#3文字以上連続を2文字に変換
def reduce_seq(text):

  result = re.sub(r'(.)\1{2,}',r'\1', text)
  return result
#二文字の除草
def reduce_w(text):

  result = re.sub(r'w{2}',r'', text)
  return result

In [ ]:
text='ここはこここはここここはこここここww'
ntext=reduce_w(text)
print(ntext)

ここはこここはここここはこここここ


## **同義語の統一**

In [ ]:
import re
import unicodedata
import nltk
from nltk.corpus import wordnet

def lemmatize_term(term, pos=None):
    if pos is None:
        synsets = wordnet.synsets(term)
        if not synsets:
            return term
        pos = synsets[0].pos()
        if pos == wordnet.ADJ_SAT:
            pos = wordnet.ADJ
    return nltk.WordNetLemmatizer().lemmatize(term, pos=pos)

## **ストップワード除去**

In [ ]:
# -*- coding: utf-8 -*-
import os
import urllib.request
from collections import Counter

from gensim import corpora


def maybe_download(path):
    url = 'http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt'
    if os.path.exists(path):
        print('File already exists.')
    else:
        print('Downloading...')
        # Download the file from `url` and save it locally under `file_name`:
        urllib.request.urlretrieve(url, path)


def create_dictionary(texts):
    dictionary = corpora.Dictionary(texts)
    return dictionary


def remove_stopwords(words, stopwords):
    words = [word for word in words if word not in stopwords]
    return words


def most_common(docs, n=100):
    fdist = Counter()
    for doc in docs:
        for word in doc:
            fdist[word] += 1
    common_words = {word for word, freq in fdist.most_common(n)}
    print('{}/{}'.format(n, len(fdist)))
    return common_words


def get_stop_words(docs, n=100, min_freq=1):
    fdist = Counter()
    for doc in docs:
        for word in doc:
            fdist[word] += 1
    common_words = {word for word, freq in fdist.most_common(n)}
    rare_words = {word for word, freq in fdist.items() if freq <= min_freq}
    stopwords = common_words.union(rare_words)
    print('{}/{}'.format(len(stopwords), len(fdist)))
    return stopwords

## **コメントAPI**

In [30]:
#コメント取得API
%cd /content/drive/My Drive/Google Colab/niconico_dataset_add_smid/TestDate
import os.path
import re
import pandas as pd
import csv

filepath = 'APIペースト用.txt'
extracted_text_array = []
pattern_prev = '">'
pattern_next = '</chat>'

extracted_text_array = []
pattern = pattern_prev + '(.*)' + pattern_next
with open(filepath) as f:
  lines = f.readlines()
  for line in lines:
    tmp_extracted_text_array = re.findall(pattern, line)
    extracted_text_array.extend(tmp_extracted_text_array)

#print(extracted_text_array)

i=0
df = pd.DataFrame(index=[], columns=['content','category'])
for row in extracted_text_array:
  df.loc[i]=[row,0]#２つ目の要素はカテゴリ
  #print(row)
  i+=1

df.head()
#データをシャッフル
#df = df.sample(frac=1, random_state=32).reset_index(drop=True)

#データ前処理
df['content'] = df['content'].map(clean_text)
df['content'] = df['content'].map(normalize)

#6文字以下の文章の除去
df['content'] = df['content'].astype('str')
mask = (df['content'].str.len() >= 7)
df = df.loc[mask]

df.to_csv('/content/drive/My Drive/Google Colab/niconico_dataset_add_smid/TestDate/君が代.tsv', sep='\t', index=False, header=None)

/content/drive/My Drive/Google Colab/niconico_dataset_add_smid/TestDate
